In [1]:
import os
%pwd

'f:\\ml projects\\chicken_disease_classification\\notebooks'

In [2]:
os.chdir('../')
%pwd

'f:\\ml projects\\chicken_disease_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelTrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation:bool
    params_image_size:list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [13]:
from chicken_disease_classifier.constants import *
from chicken_disease_classifier.utils.common import create_directories,read_yaml


class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks

        create_directories([Path(config.root_dir),Path(config.tensorboard_root_log_dir),Path(os.path.dirname(config.checkpoint_model_filepath))])

        preprare_callback_config = PrepareCallbacksConfig(
            root_dir= config.root_dir,
            tensorboard_root_log_dir= config.tensorboard_root_log_dir,
            checkpoint_model_filepath= config.checkpoint_model_filepath
        )

        return  preprare_callback_config


    def get_model_training_config(self) -> ModelTrainingConfig:
        training_config = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")

        model_training_config = ModelTrainingConfig(
            root_dir= Path(training_config.root_dir),
            trained_model_path= Path(training_config.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )
        return model_training_config



In [5]:
import tensorflow as tf
import time

In [7]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig) -> None:
        self.config = config

    @property
    def _create_tensorboard(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}")

        return tf.keras.callbacks.TensorBoard(log_dir= tb_running_log_dir)
    
    @property
    def _create_checkpoint_model(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath,save_best_only=True)
    
    def create_ckpt_callbacks(self):
        return [self._create_tensorboard,self._create_checkpoint_model]

In [19]:
class ModelTraining:
    def __init__(self,config = ModelTrainingConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle=True,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path:Path,model:tf.keras.models):
        model.save(path)

    def train_model(self,callback_list:list):
        self.steps_per_epochs = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epochs,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )    

In [20]:
try:
    config = ConfigurationManager()
    callback_config = config.get_prepare_callbacks_config()
    prepare_callback = PrepareCallback(config=callback_config)
    callback_list = prepare_callback.create_ckpt_callbacks()

    training_config = config.get_model_training_config()
    training = ModelTraining(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train_model(callback_list=callback_list)
    
except Exception as e:
    raise e

[ 2023-11-23 14:39:46,112 ] 26 - common - INFO - <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-23 14:39:46,121 ] 26 - common - INFO - <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> Loaded Successfully
[ 2023-11-23 14:39:46,123 ] 48 - common - INFO - Created Directory at : artifacts
[ 2023-11-23 14:39:46,126 ] 48 - common - INFO - Created Directory at : artifacts\prepare_callbacks
[ 2023-11-23 14:39:46,128 ] 48 - common - INFO - Created Directory at : artifacts\prepare_callbacks\tensorboard_log_dir
[ 2023-11-23 14:39:46,130 ] 48 - common - INFO - Created Directory at : artifacts\prepare_callbacks\checkpoint_dir
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
19/19 [==============================] - ETA: 0s - loss: 15.5243 - accuracy: 0.4730

f:\ml projects\chicken_disease_classification\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


19/19 [==============================] - 175s 9s/step - loss: 15.5243 - accuracy: 0.4730 - val_loss: 2.1080 - val_accuracy: 0.6875
